In [1]:
import numpy as np
import os
import PIL
import tensorflow as tf
import pathlib

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data_dir = pathlib.Path('/content/drive/MyDrive/MLPics2022/')

In [4]:
batch_size = 32
img_height = 180
img_width = 180

In [5]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=124,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

normalization_layer = layers.Rescaling(1./255)

normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
print(np.min(first_image), np.max(first_image))

class_names = train_ds.class_names
print(class_names)
num_classes = len(class_names)

model = Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


epochs=10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']


Found 286 files belonging to 8 classes.
Using 229 files for training.
Found 286 files belonging to 8 classes.
Using 57 files for validation.
0.0 1.0
['Buy_-1', 'Buy_10', 'Buy_2', 'Buy_5', 'Sell_-1', 'Sell_10', 'Sell_2', 'Sell_5']


/usr/local/lib/python3.10/dist-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 29s 3s/step - accuracy: 0.2926 - loss: 1.7361 - val_accuracy: 0.4737 - val_loss: 1.4634
Epoch 2/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - accuracy: 0.5590 - loss: 1.2945 - val_accuracy: 0.4737 - val_loss: 1.3646
Epoch 3/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step - accuracy: 0.6692 - loss: 0.9633 - val_accuracy: 0.7018 - val_loss: 1.0710
Epoch 4/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 23s 2s/step - accuracy: 0.7791 - loss: 0.6468 - val_accuracy: 0.7193 - val_loss: 0.9807
Epoch 5/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step - accuracy: 0.9114 - loss: 0.3045 - val_accuracy: 0.6491 - val_loss: 1.3942
Epoch 6/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.9248 - loss: 0.2549 - val_accuracy: 0.8070 - val_loss: 1.0852
Epoch 7/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.9784 - loss: 0.0692 - val_accuracy: 0.7895 - val_loss: 1.1522
Epoch 8/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 1.0000 - loss: 0.0218 - val_accuracy: 0.8246 - val_loss: 1.4509
Epoch 9/

In [7]:
img = tf.keras.utils.load_img(
    '/content/drive/MyDrive/Testing/57_sell_10.gif', target_size=(img_height, img_width)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print("Pic #" + str(10))

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
Pic #10
This image most likely belongs to Buy_-1 with a 72.29 percent confidence.
